In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import cm
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import math
%matplotlib inline
import seaborn as sns
import klib
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    

from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import VotingRegressor

from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.base import ClassifierMixin
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean


import datetime
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.decomposition import PCA

In [2]:
df_x_train = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
df_x_test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')
df_y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv').age
df_y_train_mer = pd.read_csv(os.path.abspath("../input")+'/y_train.csv')
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949')
IDtrain = pd.DataFrame({'custid': df_x_train.custid.unique()})
IDtest = pd.DataFrame({'custid': df_x_test.custid.unique()})

# df = pd.concat([df_x_train, df_x_test])

eda_df = pd.merge(df_x_train , df_y_train_mer, on = 'custid')

ALL = pd.read_csv(os.path.abspath("../input")+'/ALL피처추가060601.csv')

In [3]:
df = pd.read_csv(os.path.abspath("../input")+'/hyun_features')
dft = pd.read_csv(os.path.abspath("../input")+'/hyun_features_te')

In [4]:
trainid = IDtrain.custid.to_list()
X_train = ALL.query('custid in @trainid')

testid = IDtest.custid.to_list()
X_test = ALL.query('custid in @testid')

In [5]:
X_train = X_train.drop('custid', axis = 1 )

In [6]:
X_test = X_test.drop('custid', axis = 1 )

In [7]:
numeric_columns2 = X_train.dtypes[X_train.dtypes != 'object'].index.tolist()

In [8]:
# non-skew에만 처리를 하기 위해서 numeric_columns에서 skew와 non-skew로 구분
skewlist2 = []
numeric_columns_ns2 = []

for i in numeric_columns2:
    if 'SKEW' in i:
        skewlist2.append(i)
    else:
        numeric_columns_ns2.append(i)

In [9]:
# non-skew standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[numeric_columns2] = scaler.fit_transform(X_train[numeric_columns2])
X_test[numeric_columns2] = scaler.transform(X_test[numeric_columns2])

### feature concat

In [10]:
data = pd.concat([df, X_train[numeric_columns_ns2], X_train[skewlist2]], axis=1)

X_test.index = dft.index
data_te = pd.concat([dft, X_test[numeric_columns_ns2], X_test[skewlist2] ], axis=1)

In [11]:
display( data, data_te)

,1_stf,9_stf,10_stf,11_stf,12_stf,13_stf,14_stf,15_stf,16_stf,17_stf,...,행사장(여성캐주얼),행사장(여성캐쥬),행사장(잡화),화장품,식품팀,의류패션팀,인터넷백화점_y,잡화가용팀,방문당평균구매액,방문당평균구매상품수
0,0,0,0,0,2,0,0,0,0,0,...,-0.046443,-0.044738,-0.031135,1.514697,-0.346229,-0.451798,-0.006806,0.060923,0.316165,-0.391769
1,0,0,1,1,0,1,1,0,1,1,...,-0.046443,-0.044738,-0.031135,-0.380754,-0.346229,-0.076312,-0.006806,0.556603,1.848414,-0.391769
2,0,0,0,2,3,2,3,0,0,5,...,-0.046443,-0.044738,-0.031135,-0.295355,-0.310157,-0.224343,-0.006806,0.068412,-0.067703,0.702682
3,0,0,0,0,0,0,4,0,0,0,...,-0.046443,-0.044738,-0.031135,-0.559885,-0.346229,-0.118915,-0.006806,-0.559987,1.777211,0.245300
4,0,0,0,8,1,2,0,1,3,5,...,-0.046443,-0.044738,-0.031135,0.404504,0.134538,0.161607,-0.006806,0.736393,0.272373,-0.462554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,0,0,2,1,8,8,12,3,10,14,...,-0.046443,-0.044738,-0.031135,1.146022,-0.346229,5.361198,-0.006806,0.672847,1.224299,0.169069
21583,0,0,0,0,4,4,0,0,3,2,...,-0.046443,-0.044738,-0.031135,-0.601543,-0.297154,-0.476080,-0.006806,-0.073786,-0.329461,-0.160108
21584,0,0,0,0,1,3,4,7,3,2,...,-0.046443,-0.044738,-0.031135,-0.622372,0.078366,-0.422218,-0.006806,0.550462,1.160984,0.988546
21585,0,0,0,0,0,5,2,2,2,0,...,-0.046443,-0.044738,-0.031135,-0.605709,-0.346229,-0.346326,-0.006806,-0.496227,-0.559442,-0.312135


,1_stf,9_stf,10_stf,11_stf,12_stf,13_stf,14_stf,15_stf,16_stf,17_stf,...,행사장(여성캐주얼),행사장(여성캐쥬),행사장(잡화),화장품,식품팀,의류패션팀,인터넷백화점_y,잡화가용팀,방문당평균구매액,방문당평균구매상품수
0,0,0,0,0,0,0,0,0,3,15,...,-0.046443,-0.044738,-0.031135,-0.570299,0.266301,-0.017519,-0.006806,0.486510,1.207685,1.285845
1,0,0,1,11,18,11,8,9,9,15,...,-0.046443,-0.044738,-0.031135,3.468470,-0.140063,1.420423,-0.006806,0.891128,0.473286,2.500668
2,0,0,0,0,4,2,12,2,1,9,...,-0.046443,-0.044738,-0.031135,-0.464695,-0.253084,-0.165801,-0.006806,-0.393869,-0.714909,0.066920
3,0,0,0,0,0,0,0,0,0,0,...,-0.046443,-0.044738,-0.031135,-0.566133,-0.346229,-0.553782,-0.006806,-0.561271,-0.708207,-1.241194
4,0,0,0,0,0,0,0,1,1,0,...,-0.046443,-0.044738,-0.031135,-0.366174,-0.346229,-0.490869,-0.006806,-0.520191,-0.618107,-0.497947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,0,0,0,0,3,0,0,0,0,0,...,-0.046443,-0.044738,-0.031135,-0.480734,-0.346229,-0.344074,-0.006806,-0.518479,0.674657,0.245300
14376,0,0,0,0,0,0,0,1,0,0,...,-0.046443,-0.044738,-0.031135,-0.241199,-0.346229,-0.553782,-0.006806,-0.494515,0.124715,-1.241194
14377,0,0,0,0,0,0,0,0,0,1,...,-0.046443,-0.044738,-0.031135,-0.574465,-0.346229,-0.479611,-0.006806,-0.562982,-0.422558,-1.241194
14378,0,0,0,0,0,0,0,0,1,2,...,-0.046443,-0.044738,-0.031135,-0.539055,-0.272087,-0.553782,-0.006806,-0.547149,-0.248877,3.218286


In [12]:
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949').age
data.shape, data_te.shape, y_train.shape

((21587, 5098), (14380, 5098), (21587,))

In [13]:
data.columns = data.columns.astype(str)
data_te.columns = data_te.columns.astype(str)

### Modeling

In [15]:
data2 = data.values
data_te2 = data_te.values

In [16]:
ftr = data2[:, 1:]
target = y_train
target_log = np.log1p(target)

In [17]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr, target, test_size=0.2 , stratify = target , random_state=0)
train_x.shape, valid_x.shape

((17269, 5097), (4318, 5097))

In [18]:
from lightgbm import LGBMRegressor

clf = LGBMRegressor(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric = 'RMSE', 
        verbose=100, early_stopping_rounds= 50)

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 8.08188	training's l2: 65.3168	valid_1's rmse: 8.65417	valid_1's l2: 74.8947
[200]	training's rmse: 7.3421	training's l2: 53.9065	valid_1's rmse: 8.38242	valid_1's l2: 70.265
[300]	training's rmse: 6.84944	training's l2: 46.9148	valid_1's rmse: 8.30163	valid_1's l2: 68.917
[400]	training's rmse: 6.46629	training's l2: 41.8129	valid_1's rmse: 8.28015	valid_1's l2: 68.5609
[500]	training's rmse: 6.1421	training's l2: 37.7254	valid_1's rmse: 8.27539	valid_1's l2: 68.482
[600]	training's rmse: 5.85635	training's l2: 34.2968	valid_1's rmse: 8.26558	valid_1's l2: 68.3199
[700]	training's rmse: 5.60838	training's l2: 31.4539	valid_1's rmse: 8.26459	valid_1's l2: 68.3034
Early stopping, best iteration is:
[665]	training's rmse: 5.69307	training's l2: 32.411	valid_1's rmse: 8.2629	valid_1's l2: 68.2756


LGBMRegressor(learning_rate=0.02, max_depth=12, n_estimators=1000,
              num_leaves=32, silent=-1, subsample=0.8, verbose=-1)

#### Feature Selection & Cross Validation

In [31]:
ftr = data2[:, 1:]
target = y_train
target_log = np.log1p(target)

In [32]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(clf, threshold='2.0*mean')
smf.fit(ftr, target)
X_new = smf.transform(ftr)
X_te_new = smf.transform(data_te2[:, 1:])
feature_idx = smf.get_support()
# feature_name = ftr.columns[feature_idx]

In [33]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_new, target, scoring='neg_mean_squared_error', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))
print('RMSE:', np.sqrt(-np.mean(scores)))
# mean -
# 2.0mean - 8.199613557837917
# 3.0mean - 8.207348627039885
# 3.5mean -       
# 4.0mean - 
# 4.5mean - 
# 5.0mean - 
# 6.0mean - 
# 6.5mean - 
# 7.0mean - 8.225485195696558


교차 검증별 정확도: [-65.858  -68.8208 -69.7267 -66.2238 -65.539 ]
평균 검증 정확도: -67.2337
RMSE: 8.199613557837917


In [78]:
X_new.shape, X_te_new.shape

((21587, 581), (14380, 581))

#### Hyper parameter Tuning_lgbm

In [40]:
# X_new = np.array(data)
# X_te_new = np.array(data_te)

In [35]:
ftr = X_new
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949').age
target = y_train
target_log = np.log1p(target)

In [36]:
target.shape

(21587,)

In [37]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr, target, test_size=0.3, stratify = target,  random_state=1000)
train_x.shape, valid_x.shape

((15110, 581), (6477, 581))

In [38]:
def get_rmse(model):
    pred = model.predict(valid_x)
    mse = mean_squared_error(valid_y , pred)
    rmse = np.sqrt(mse)
    print('{0} 로그 변환된 RMSE: {1}'.format(model.__class__.__name__,np.round(rmse, 3)))
    return rmse

def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses



lgbm = LGBMRegressor(random_state = 1000  )
lgbm = lgbm.fit(train_x , train_y)

cat = CatBoostRegressor(random_state=1000 )
cat = cat.fit(train_x , train_y)

xgb = XGBRegressor(random_state = 1000 )
xgb.fit(train_x , train_y )

reg_ridge = Ridge(random_state = 1000)
reg_ridge.fit(train_x, train_y)


models = [lgbm, cat, xgb, reg_ridge ]
get_rmses(models)




# LGBMRegressor 로그 변환된 RMSE: 8.318
# CatBoostRegressor 로그 변환된 RMSE: 8.227
# XGBRegressor 로그 변환된 RMSE: 8.778
# Ridge 로그 변환된 RMSE: 8.469

Learning rate set to 0.063286
0:	learn: 10.2967141	total: 263ms	remaining: 4m 22s
1:	learn: 10.1562030	total: 340ms	remaining: 2m 49s
2:	learn: 10.0385337	total: 415ms	remaining: 2m 17s
3:	learn: 9.9386690	total: 486ms	remaining: 2m 1s
4:	learn: 9.8401474	total: 562ms	remaining: 1m 51s
5:	learn: 9.7422273	total: 642ms	remaining: 1m 46s
6:	learn: 9.6557683	total: 714ms	remaining: 1m 41s
7:	learn: 9.5755614	total: 791ms	remaining: 1m 38s
8:	learn: 9.5052347	total: 860ms	remaining: 1m 34s
9:	learn: 9.4384202	total: 933ms	remaining: 1m 32s
10:	learn: 9.3789410	total: 1.02s	remaining: 1m 32s
11:	learn: 9.3243263	total: 1.09s	remaining: 1m 30s
12:	learn: 9.2736793	total: 1.16s	remaining: 1m 28s
13:	learn: 9.2265025	total: 1.22s	remaining: 1m 26s
14:	learn: 9.1774537	total: 1.29s	remaining: 1m 24s
15:	learn: 9.1358822	total: 1.35s	remaining: 1m 23s
16:	learn: 9.0936508	total: 1.42s	remaining: 1m 21s
17:	learn: 9.0563088	total: 1.48s	remaining: 1m 20s
18:	learn: 9.0246515	total: 1.55s	remainin

159:	learn: 7.8196229	total: 12.1s	remaining: 1m 3s
160:	learn: 7.8157770	total: 12.2s	remaining: 1m 3s
161:	learn: 7.8114588	total: 12.2s	remaining: 1m 3s
162:	learn: 7.8064549	total: 12.3s	remaining: 1m 3s
163:	learn: 7.8022473	total: 12.4s	remaining: 1m 3s
164:	learn: 7.7987167	total: 12.4s	remaining: 1m 2s
165:	learn: 7.7940946	total: 12.5s	remaining: 1m 2s
166:	learn: 7.7899281	total: 12.6s	remaining: 1m 2s
167:	learn: 7.7853964	total: 12.6s	remaining: 1m 2s
168:	learn: 7.7812122	total: 12.7s	remaining: 1m 2s
169:	learn: 7.7771911	total: 12.8s	remaining: 1m 2s
170:	learn: 7.7718252	total: 12.8s	remaining: 1m 2s
171:	learn: 7.7669960	total: 12.9s	remaining: 1m 2s
172:	learn: 7.7627816	total: 13s	remaining: 1m 2s
173:	learn: 7.7551655	total: 13s	remaining: 1m 1s
174:	learn: 7.7510262	total: 13.1s	remaining: 1m 1s
175:	learn: 7.7476177	total: 13.2s	remaining: 1m 1s
176:	learn: 7.7436976	total: 13.3s	remaining: 1m 1s
177:	learn: 7.7410384	total: 13.3s	remaining: 1m 1s
178:	learn: 7.73

320:	learn: 7.1542523	total: 22.8s	remaining: 48.2s
321:	learn: 7.1491706	total: 22.8s	remaining: 48.1s
322:	learn: 7.1463301	total: 22.9s	remaining: 48s
323:	learn: 7.1423775	total: 23s	remaining: 47.9s
324:	learn: 7.1385353	total: 23s	remaining: 47.8s
325:	learn: 7.1346226	total: 23.1s	remaining: 47.7s
326:	learn: 7.1310698	total: 23.1s	remaining: 47.6s
327:	learn: 7.1276832	total: 23.2s	remaining: 47.5s
328:	learn: 7.1231868	total: 23.3s	remaining: 47.5s
329:	learn: 7.1183426	total: 23.3s	remaining: 47.4s
330:	learn: 7.1142393	total: 23.4s	remaining: 47.3s
331:	learn: 7.1119108	total: 23.5s	remaining: 47.3s
332:	learn: 7.1067349	total: 23.6s	remaining: 47.3s
333:	learn: 7.1021134	total: 23.7s	remaining: 47.3s
334:	learn: 7.0986629	total: 23.8s	remaining: 47.3s
335:	learn: 7.0952618	total: 23.9s	remaining: 47.3s
336:	learn: 7.0906471	total: 24.1s	remaining: 47.3s
337:	learn: 7.0874385	total: 24.2s	remaining: 47.3s
338:	learn: 7.0847352	total: 24.3s	remaining: 47.3s
339:	learn: 7.0826

479:	learn: 6.6516809	total: 36.1s	remaining: 39.1s
480:	learn: 6.6475658	total: 36.2s	remaining: 39s
481:	learn: 6.6448957	total: 36.2s	remaining: 38.9s
482:	learn: 6.6417959	total: 36.3s	remaining: 38.9s
483:	learn: 6.6389116	total: 36.4s	remaining: 38.8s
484:	learn: 6.6344862	total: 36.5s	remaining: 38.8s
485:	learn: 6.6312786	total: 36.6s	remaining: 38.7s
486:	learn: 6.6281563	total: 36.7s	remaining: 38.6s
487:	learn: 6.6253159	total: 36.7s	remaining: 38.6s
488:	learn: 6.6226224	total: 36.8s	remaining: 38.5s
489:	learn: 6.6201802	total: 36.9s	remaining: 38.4s
490:	learn: 6.6168062	total: 37s	remaining: 38.3s
491:	learn: 6.6133317	total: 37s	remaining: 38.2s
492:	learn: 6.6097341	total: 37.1s	remaining: 38.2s
493:	learn: 6.6052643	total: 37.2s	remaining: 38.1s
494:	learn: 6.6030386	total: 37.3s	remaining: 38s
495:	learn: 6.6005186	total: 37.3s	remaining: 37.9s
496:	learn: 6.5974447	total: 37.4s	remaining: 37.9s
497:	learn: 6.5945537	total: 37.5s	remaining: 37.8s
498:	learn: 6.591251

638:	learn: 6.2113450	total: 48.6s	remaining: 27.4s
639:	learn: 6.2084765	total: 48.7s	remaining: 27.4s
640:	learn: 6.2059248	total: 48.7s	remaining: 27.3s
641:	learn: 6.2034632	total: 48.8s	remaining: 27.2s
642:	learn: 6.1991501	total: 48.9s	remaining: 27.2s
643:	learn: 6.1972107	total: 49s	remaining: 27.1s
644:	learn: 6.1948107	total: 49.1s	remaining: 27s
645:	learn: 6.1923554	total: 49.2s	remaining: 26.9s
646:	learn: 6.1900123	total: 49.2s	remaining: 26.9s
647:	learn: 6.1861770	total: 49.3s	remaining: 26.8s
648:	learn: 6.1831728	total: 49.4s	remaining: 26.7s
649:	learn: 6.1807528	total: 49.5s	remaining: 26.7s
650:	learn: 6.1784525	total: 49.6s	remaining: 26.6s
651:	learn: 6.1756294	total: 49.7s	remaining: 26.5s
652:	learn: 6.1735468	total: 49.8s	remaining: 26.4s
653:	learn: 6.1700558	total: 49.8s	remaining: 26.4s
654:	learn: 6.1676225	total: 49.9s	remaining: 26.3s
655:	learn: 6.1641378	total: 50s	remaining: 26.2s
656:	learn: 6.1610659	total: 50.1s	remaining: 26.1s
657:	learn: 6.1587

799:	learn: 5.8121913	total: 1m 1s	remaining: 15.3s
800:	learn: 5.8098410	total: 1m 1s	remaining: 15.2s
801:	learn: 5.8072153	total: 1m 1s	remaining: 15.1s
802:	learn: 5.8044351	total: 1m 1s	remaining: 15.1s
803:	learn: 5.8013377	total: 1m 1s	remaining: 15s
804:	learn: 5.8005849	total: 1m 1s	remaining: 14.9s
805:	learn: 5.7981549	total: 1m 1s	remaining: 14.8s
806:	learn: 5.7954295	total: 1m 1s	remaining: 14.7s
807:	learn: 5.7932843	total: 1m 1s	remaining: 14.7s
808:	learn: 5.7910972	total: 1m 1s	remaining: 14.6s
809:	learn: 5.7885026	total: 1m 1s	remaining: 14.5s
810:	learn: 5.7860838	total: 1m 1s	remaining: 14.4s
811:	learn: 5.7839538	total: 1m 1s	remaining: 14.4s
812:	learn: 5.7799160	total: 1m 2s	remaining: 14.3s
813:	learn: 5.7778329	total: 1m 2s	remaining: 14.2s
814:	learn: 5.7754887	total: 1m 2s	remaining: 14.1s
815:	learn: 5.7728781	total: 1m 2s	remaining: 14s
816:	learn: 5.7698904	total: 1m 2s	remaining: 14s
817:	learn: 5.7675910	total: 1m 2s	remaining: 13.9s
818:	learn: 5.7653

958:	learn: 5.4489368	total: 1m 12s	remaining: 3.08s
959:	learn: 5.4487334	total: 1m 12s	remaining: 3.01s
960:	learn: 5.4478060	total: 1m 12s	remaining: 2.93s
961:	learn: 5.4464685	total: 1m 12s	remaining: 2.85s
962:	learn: 5.4437075	total: 1m 12s	remaining: 2.78s
963:	learn: 5.4416717	total: 1m 12s	remaining: 2.71s
964:	learn: 5.4389695	total: 1m 12s	remaining: 2.63s
965:	learn: 5.4373379	total: 1m 12s	remaining: 2.55s
966:	learn: 5.4347530	total: 1m 12s	remaining: 2.48s
967:	learn: 5.4321529	total: 1m 12s	remaining: 2.4s
968:	learn: 5.4306282	total: 1m 12s	remaining: 2.33s
969:	learn: 5.4283440	total: 1m 12s	remaining: 2.25s
970:	learn: 5.4255050	total: 1m 12s	remaining: 2.18s
971:	learn: 5.4236846	total: 1m 13s	remaining: 2.1s
972:	learn: 5.4217671	total: 1m 13s	remaining: 2.03s
973:	learn: 5.4196239	total: 1m 13s	remaining: 1.95s
974:	learn: 5.4178898	total: 1m 13s	remaining: 1.88s
975:	learn: 5.4159155	total: 1m 13s	remaining: 1.8s
976:	learn: 5.4132720	total: 1m 13s	remaining: 1.

[8.317722126952988, 8.226987560575429, 8.77767236248439, 8.469065002823504]

In [39]:
from bayes_opt import BayesianOptimization

In [40]:
bayesian_params = {
    'max_depth':(8, 16),
    'num_leaves':(24, 64),
    'min_child_samples':(10, 200),
    'min_child_weight':(1, 50),
    'subsample':(0.5, 1),
    'colsample_bytree':(0.5, 1),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha':(0.01, 50)
}

In [41]:
def lgb_rmse_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                colsample_bytree, max_bin, reg_lambda, reg_alpha):
    
    params = {
        "n_estimators":2000, 
        "learning_rate":0.02,
        'max_depth':int(round(max_depth)),
        'num_leaves':int(round(num_leaves)),
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample':max(min(subsample, 1), 0),
        'colsample_bytree':max(min(colsample_bytree, 1), 0),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0)
    }
    
    lgb_model = LGBMRegressor(**params)
    lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'RMSE', verbose= 100, 
                early_stopping_rounds= 100)
    valid_pred = lgb_model.predict(valid_x)
    RMSE = np.sqrt(mean_squared_error(valid_y, valid_pred))
    
    return RMSE

In [42]:
lgbBO = BayesianOptimization(f=lgb_rmse_eval, pbounds=bayesian_params, random_state=1000)
lgbBO.maximize(init_points=5, n_iter=25)

|   iter    |  target   | colsam... |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 8.07906	training's l2: 65.2712	valid_1's rmse: 8.59034	valid_1's l2: 73.794
[200]	training's rmse: 7.35293	training's l2: 54.0656	valid_1's rmse: 8.32396	valid_1's l2: 69.2883
[300]	training's rmse: 6.88104	training's l2: 47.3488	valid_1's rmse: 8.24524	valid_1's l2: 67.9841
[400]	training's rmse: 6.49982	training's l2: 42.2477	valid_1's rmse: 8.21169	valid_1's l2: 67.4318
[500]	training's rmse: 6.17983	training's l2: 38.1904	valid_1's rmse: 8.19794	valid_1's l2: 67.2062
[600]	training's rmse: 5.89184	training's l2: 34.7137	valid_1's rmse: 8.18785	valid_1's l2: 67.0409
[700]	training's rmse: 5.6305	training's l2: 31.7025	valid_1's rmse: 8.

[700]	training's rmse: 5.89944	training's l2: 34.8035	valid_1's rmse: 8.19587	valid_1's l2: 67.1723
[800]	training's rmse: 5.67307	training's l2: 32.1838	valid_1's rmse: 8.19504	valid_1's l2: 67.1587
Early stopping, best iteration is:
[780]	training's rmse: 5.71737	training's l2: 32.6883	valid_1's rmse: 8.19454	valid_1's l2: 67.1504
|  7        |  8.195    |  0.6744   |  384.4    |  15.72    |  183.3    |  44.39    |  29.63    |  5.094    |  4.801    |  0.8988   |
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 8.0218	training's l2: 64.3494	valid_1's rmse: 8.60019	valid_1's l2: 73.9632
[200]	training's rmse: 7.25613	training's l2: 52.6514	valid_1's rmse: 8.33148	valid_1's l2: 69.4136
[300]	training's rmse: 6.75011	training's l2: 45.564	valid_1's rmse: 8.25439	valid_1's l2: 68.135
[400]	training's rmse: 6.3339	training's l2: 40.1183	valid_1's rmse: 8.22237	valid_1's l2: 67.6073
[500]	training's rmse: 5.98383	training's l2: 35.8062	valid_1's rmse: 8.2

[400]	training's rmse: 5.63818	training's l2: 31.789	valid_1's rmse: 8.20878	valid_1's l2: 67.3841
[500]	training's rmse: 5.20698	training's l2: 27.1126	valid_1's rmse: 8.2005	valid_1's l2: 67.2481
[600]	training's rmse: 4.8266	training's l2: 23.296	valid_1's rmse: 8.19269	valid_1's l2: 67.1201
[700]	training's rmse: 4.48442	training's l2: 20.11	valid_1's rmse: 8.19547	valid_1's l2: 67.1657
Early stopping, best iteration is:
[664]	training's rmse: 4.60342	training's l2: 21.1915	valid_1's rmse: 8.19151	valid_1's l2: 67.1008
|  14       |  8.192    |  0.8278   |  110.7    |  15.82    |  82.63    |  9.746    |  54.33    |  9.338    |  4.784    |  0.9855   |
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.8068	training's l2: 60.9462	valid_1's rmse: 8.55667	valid_1's l2: 73.2165
[200]	training's rmse: 6.91888	training's l2: 47.8709	valid_1's rmse: 8.30707	valid_1's l2: 69.0074
[300]	training's rmse: 6.31746	training's l2: 39.9104	valid_1's rmse: 8.2399

[400]	training's rmse: 6.1061	training's l2: 37.2844	valid_1's rmse: 8.21495	valid_1's l2: 67.4854
[500]	training's rmse: 5.72946	training's l2: 32.8267	valid_1's rmse: 8.20387	valid_1's l2: 67.3035
[600]	training's rmse: 5.39465	training's l2: 29.1022	valid_1's rmse: 8.19933	valid_1's l2: 67.229
[700]	training's rmse: 5.09083	training's l2: 25.9166	valid_1's rmse: 8.19551	valid_1's l2: 67.1664
[800]	training's rmse: 4.82004	training's l2: 23.2328	valid_1's rmse: 8.19473	valid_1's l2: 67.1536
Early stopping, best iteration is:
[748]	training's rmse: 4.9596	training's l2: 24.5976	valid_1's rmse: 8.19355	valid_1's l2: 67.1343
|  21       |  8.194    |  0.6121   |  56.01    |  13.32    |  86.02    |  2.535    |  43.43    |  12.8     |  3.448    |  0.5243   |
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.88026	training's l2: 62.0984	valid_1's rmse: 8.56472	valid_1's l2: 73.3544
[200]	training's rmse: 7.04845	training's l2: 49.6807	valid_1's rmse: 8.

[500]	training's rmse: 5.84683	training's l2: 34.1855	valid_1's rmse: 8.19437	valid_1's l2: 67.1477
[600]	training's rmse: 5.52279	training's l2: 30.5012	valid_1's rmse: 8.18921	valid_1's l2: 67.0631
[700]	training's rmse: 5.22589	training's l2: 27.31	valid_1's rmse: 8.18622	valid_1's l2: 67.0142
[800]	training's rmse: 4.9596	training's l2: 24.5976	valid_1's rmse: 8.18875	valid_1's l2: 67.0556
Early stopping, best iteration is:
[713]	training's rmse: 5.19134	training's l2: 26.95	valid_1's rmse: 8.18594	valid_1's l2: 67.0096
|  28       |  8.186    |  0.5543   |  266.6    |  12.65    |  147.4    |  13.2     |  45.91    |  23.86    |  8.469    |  0.9726   |
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.93758	training's l2: 63.0052	valid_1's rmse: 8.58069	valid_1's l2: 73.6282
[200]	training's rmse: 7.12689	training's l2: 50.7926	valid_1's rmse: 8.31648	valid_1's l2: 69.1639
[300]	training's rmse: 6.60159	training's l2: 43.581	valid_1's rmse: 8.242

In [43]:
lgbBO.res

[{'target': 8.182122940865398,
  'params': {'colsample_bytree': 0.8267947927323047,
   'max_bin': 66.35340213095881,
   'max_depth': 15.602262914792195,
   'min_child_samples': 101.61636627131966,
   'min_child_weight': 43.75125222391973,
   'num_leaves': 32.4933072369088,
   'reg_alpha': 2.045074142206765,
   'reg_lambda': 3.9725474189957124,
   'subsample': 0.61656609867419}},
 {'target': 8.197623319281618,
  'params': {'colsample_bytree': 0.9208703621265308,
   'max_bin': 111.47034874950597,
   'max_depth': 13.93975626865927,
   'min_child_samples': 84.50928428367985,
   'min_child_weight': 9.930569477666833,
   'num_leaves': 53.7415765836856,
   'reg_alpha': 3.488408227688028,
   'reg_lambda': 8.853486706603126,
   'subsample': 0.9763221996107709}},
 {'target': 8.22208704283251,
  'params': {'colsample_bytree': 0.9655717173387555,
   'max_bin': 213.56116699200484,
   'max_depth': 8.23185327509497,
   'min_child_samples': 196.58522207365755,
   'min_child_weight': 17.64224649812914,

In [44]:
# dictionary에 있는 target값을 모두 추출
target_list = []
for result in lgbBO.res:
    target = result['target']
    target_list.append(target)
print(target_list)
# 가장 큰 target 값을 가지는 순번(index)를 추출
print('maximum target index:', np.argmin(np.array(target_list)))

[8.182122940865398, 8.197623319281618, 8.22208704283251, 8.219204451219671, 8.203543721101621, 8.196010825482142, 8.194536759035888, 8.185851778514591, 8.181840547348232, 8.20013517309769, 8.172892840706846, 8.181618598585732, 8.187217277548443, 8.191510637999292, 8.198231394860912, 8.214368607895045, 8.205552464129399, 8.200756520495782, 8.197481069653549, 8.206754863699791, 8.193554223877586, 8.200109944431139, 8.205414929163604, 8.190244985724913, 8.203819786344848, 8.191310306440302, 8.172484666063644, 8.185938050237052, 8.186104563894819, 8.183877103368667]
maximum target index: 26


In [45]:
# 가장 큰 target값을 가지는 index값을 기준으로 res에서 해당 parameter 추출. 
max_dict = lgbBO.res[np.argmin(np.array(target_list))]
print(max_dict)

{'target': 8.172484666063644, 'params': {'colsample_bytree': 0.5948136651099992, 'max_bin': 65.54879996738464, 'max_depth': 13.147136974234437, 'min_child_samples': 102.1230118149357, 'min_child_weight': 8.454065086716847, 'num_leaves': 37.71286374411538, 'reg_alpha': 8.495603759047018, 'reg_lambda': 8.3932459613762, 'subsample': 0.7136850800625171}}


In [47]:
ftr = DataFrame(X_new)
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv', encoding='cp949').age
target = y_train
target_log = np.log1p(target)

In [48]:
from sklearn.model_selection import KFold

def train_apps_all_with_oof(ftr, target, nfolds=5):
    ftr = ftr
    target = target

    # nfolds 개의 cross validatin fold set을 가지는 KFold 생성 
    folds = KFold(n_splits=nfolds, shuffle=True, random_state=0)
    
    # Out of Folds로 학습된 모델의 validation set을 예측하여 결과 확률을 담을 array 생성.
    # validation set가 n_split갯수만큼 있으므로 크기는 ftr_app의 크기가 되어야 함. 
    oof_preds = np.zeros((ftr.shape[0],))  
    
    # Ouf of Folds로 학습된 모델의 test dataset을 예측하여 결과 확률을 담을 array 생성. 
    test_preds = np.zeros(((X_te_new.shape[0],)))
    
    # n_estimators를 4000까지 확대. 
    clf = LGBMRegressor(
                nthread=4,
                n_estimators=4000,
                learning_rate=0.01,
                max_depth=13,
                num_leaves=37,
                colsample_bytree=0.594,
                subsample=0.713,
                max_bin= 65,
                reg_alpha=8.495,
                reg_lambda=8.393,
                min_child_weight=8,
                min_child_samples=102,
                silent=-1,
                verbose=-1,
                )

    # nfolds 번 cross validation Iteration 반복하면서 OOF 방식으로 학습 및 테스트 데이터 예측
    for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(ftr)):
        print('##### iteration ', fold_idx, ' 시작')
        # 학습용 데이터 세트의 인덱스와 검증용 데이터 세트의 인덱스 추출하여 이를 기반으로 학습/검증 데이터 추출
        train_x = ftr.iloc[train_idx, :]
        train_y = target.iloc[train_idx]
        valid_x = ftr.iloc[valid_idx, :]
        valid_y = target.iloc[valid_idx]
        
        # 추출된 학습/검증 데이터 세트로 모델 학습. early_stopping은 200으로 증가. 
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'RMSE', verbose= 200, 
                early_stopping_rounds= 200)
        # 검증 데이터 세트로 예측된 확률 저장. 사용되지는 않음. 
        #oof_preds[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration_)       
        # 학습된 모델로 테스트 데이터 세트에 예측 확률 계산. 
        # nfolds 번 반복 실행하므로 평균 확률을 구하기 위해 개별 수행시 마다 수행 횟수로 나눈 확률을 추후에 더해서 최종 평균 확률 계산. 
        test_preds += clf.predict(X_te_new, num_iteration=clf.best_iteration_)/folds.n_splits
        
        
    return clf, test_preds

In [49]:
clf, test_preds = train_apps_all_with_oof(ftr, target, nfolds=5)

##### iteration  0  시작
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 8.09925	training's l2: 65.5978	valid_1's rmse: 8.6051	valid_1's l2: 74.0477
[400]	training's rmse: 7.39934	training's l2: 54.7503	valid_1's rmse: 8.32582	valid_1's l2: 69.3192
[600]	training's rmse: 6.94931	training's l2: 48.2929	valid_1's rmse: 8.24741	valid_1's l2: 68.0197
[800]	training's rmse: 6.58443	training's l2: 43.3547	valid_1's rmse: 8.21011	valid_1's l2: 67.406
[1000]	training's rmse: 6.27295	training's l2: 39.3498	valid_1's rmse: 8.19383	valid_1's l2: 67.1389
[1200]	training's rmse: 5.99461	training's l2: 35.9353	valid_1's rmse: 8.18104	valid_1's l2: 66.9294
[1400]	training's rmse: 5.74255	training's l2: 32.9769	valid_1's rmse: 8.17517	valid_1's l2: 66.8335
[1600]	training's rmse: 5.51174	training's l2: 30.3793	valid_1's rmse: 8.1713	valid_1's l2: 66.7701
[

In [50]:
test_preds

array([36.48608066, 41.57529689, 26.88872272, ..., 35.15863906,
       33.5709692 , 27.06398132])

In [51]:
IDtest['age'] = test_preds ; sub = IDtest

In [52]:
sub.to_csv('submissions_0613_hyun_ha_scaled_lgbm_tun.csv', index=False)